In [136]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import time
%matplotlib inline

---

In [137]:
#DBM data below

In [138]:
DBM = pd.DataFrame()
for file in glob.glob("DBM*.csv"):
    df = pd.read_csv(file)
    df = df[df["Advertiser Currency"].isnull() == False]
    DBM = DBM.append(df, ignore_index=True)

In [139]:
DBM = DBM.loc[:, ["Month", "Advertiser", "Public Inventory", "Impressions", 
                  "Clicks", "Revenue (Adv Currency)", "Media Cost (Advertiser Currency)", "Advertiser Currency"]]

In [140]:
DBM.columns = ["Month", "Advertiser", "Public", "Impressions", "Clicks", "Spend", "Media", "Currency"]

In [141]:
DBM["DSP"] = "DBM"
DBM["Type"] = "General"

---

In [142]:
#YouTube/TrueView data below

In [143]:
YT = pd.DataFrame()
for file in glob.glob("YT*.csv"):
    df = pd.read_csv(file)
    df = df[df["Advertiser Currency"].isnull() == False]
    YT = YT.append(df, ignore_index=True)

In [144]:
YT["Public"] = "N/A"
YT["DSP"] = "DBM"
YT["Type"] = "TrueView"

In [145]:
YT = YT.loc[:, ["Month", "Advertiser", "Public","Impressions", 
                "Clicks", "Revenue (Adv Currency)", "Media Cost (Advertiser Currency)", "Advertiser Currency", "DSP", "Type"]]

In [146]:
YT.columns = ['Month', 'Advertiser', 'Public', 'Impressions', 'Clicks',
       'Spend', 'Media', 'Currency', 'DSP', 'Type']

---

In [147]:
#Videology data below
#please note the data in the template the time frame please select the specific time rather than 'last month'

In [148]:
videology = pd.DataFrame()
for file in glob.glob("Videology*.csv"):
    df = pd.read_csv(file)
    df = df[df["Placement ID"].isnull() == False]
    videology = videology.append(df, ignore_index=True)

In [149]:
videology.columns = ["Advertiser", "Placement", "PlacementName", "Date", "Impressions", "Spend", "Clicks"]

In [150]:
list=[]
for month in videology.Date:
    g = month[-4:] + "/" + "{0:0=2d}".format(time.strptime(month[:3], "%b").tm_mon)
    list.append(g)

In [151]:
videology.Impressions = videology.Impressions.apply(lambda x: float(x.split()[0].replace(',', '')))
videology.Clicks = videology.Clicks.apply(lambda x: float(x.split()[0].replace(',', '')))
videology.Spend = videology.Spend.apply(lambda x: float(x.split()[0].replace(',', '')))

In [152]:
videology["Currency"] = "GBP"
videology["DSP"] = "Videology"
videology["Type"] = "General"
videology.loc[(videology.PlacementName.str.contains("Channel 4")) | 
              (videology.PlacementName.str.contains("C4")), "Type"] = "Channel 4"
videology["Media"] = videology.Spend * 0.7
videology.loc[videology.Type == "Channel 4", "Media"] = videology.Spend * 0.8
videology["Public"] = "N/A"
videology["Month"] = list

In [153]:
videology = videology.loc[:, ['Month', 'Advertiser', 'Public', 'Impressions', 'Clicks', 'Spend',
       'Media', 'Currency', 'DSP', 'Type']]

---

In [154]:
#TheTradeDesk data below

In [155]:
TTD = pd.DataFrame()
for file in glob.glob("TTD*.tsv"):
    df = pd.read_csv(file)
    TTD = TTD.append(df, ignore_index=True)
TTD.columns = ["all"]
TTD = pd.DataFrame(TTD["all"].str.split("\t").tolist(), 
             columns=["Date", "Advertiser","Currency", "PMP", "Impressions", "Clicks", "Spend", "Media"])

In [156]:
TTD["Month"] = TTD.Date.str.split("/").str[2] + "/" + TTD.Date.str.split("/").str[1]
TTD["Public"] = "Yes"
TTD.loc[TTD.PMP.str.contains("PMP"), "Public"] = "No"
TTD["DSP"] = "TTD"
TTD["Type"] = "General"

In [157]:
TTD = TTD.loc[:, ["Month", "Advertiser", "Public", "Impressions", "Clicks", "Spend", "Media", "Currency", "DSP", "Type"]]

In [158]:
TTD.Impressions = TTD.Impressions.astype("float64")
TTD.Clicks = TTD.Clicks.astype("float64")
TTD.Spend = TTD.Spend.astype("float64")
TTD.Media = TTD.Media.astype("float64")

In [159]:
#below is fixed for 2016 TTD data 

In [160]:
TTD_old = pd.read_csv("TTD_0116_1216.csv")

In [161]:
TTD_old["Month"] = TTD_old.Date.str.split("/").str[2] + "/" + TTD_old.Date.str.split("/").str[1]
TTD_old["Public"] = "Yes"
TTD_old.loc[TTD_old["Inventory Contract"] == "PMP", "Public"] = "No"
TTD_old["DSP"] = "TTD"
TTD_old["Type"] = "General"

In [162]:
TTD_old = TTD_old.loc[:, ["Month", "Advertiser", "Public", "Impressions", 
                          "Clicks", "Advertiser Cost (Adv Currency)", "Media Cost (Adv Currency)", 
                          "Advertiser Currency Code", "DSP", "Type"]]

In [163]:
TTD_old.columns = ['Month', 'Advertiser', 'Public', 'Impressions', 'Clicks',
       'Spend', 'Media','Currency', 'DSP', 'Type']

In [164]:
TTD = TTD.append(TTD_old, ignore_index=True)

---

In [165]:
#RocketFuel data below

In [166]:
RF = pd.DataFrame()
for file in glob.glob("RF_0117_0417.csv"):
    df = pd.read_csv(file, header=2)
    RF = RF.append(df, ignore_index=True)
RF.Spend = RF.Spend.str.replace("GBP ", "")
RF["Media Cost"] = RF["Media Cost"].str.replace("GBP ", "")

In [167]:
RF["Public"] = "Yes"
RF.loc[RF["Exchange Deal ID"].isnull() == False, "Public"] = "No"
RF["Month"] = RF["Year and Month"].str.split("-").str[0] + "/" + RF["Year and Month"].str.split("-").str[1]
RF["DSP"] = "RocketFuel"
RF["Type"] = "General"
RF["Currency"] = "GBP"

In [168]:
RF = RF.loc[:, ["Month", "Advertiser", "Public", "Imps", "Clicks", "Spend", "Media Cost", "Currency", "DSP", "Type"]]

In [169]:
RF.columns = ['Month', 'Advertiser', 'Public', 'Impressions', 'Clicks', 'Spend', 'Media', 'Currency', 'DSP', 'Type']

In [170]:
RF.Spend = RF.Spend.astype("float64")
RF.Media = RF.Media.astype("float64")

---

In [171]:
#Data from all DSPs

In [172]:
data = DBM.append([TTD, YT, videology, RF], ignore_index=True)

In [173]:
np.unique(data.Currency)
USDtoGBP = 0.77
EURtoGBP = 0.84

In [174]:
data["cSpend"] = data.Spend 
data.loc[data.Currency == "USD","cSpend"] = data.Spend * USDtoGBP
data.loc[data.Currency == "EUR","cSpend"] = data.Spend * EURtoGBP

---

In [175]:
#file to ZO folder on desktop

In [176]:
data.sort_values("Month", ascending=True).to_csv("D:\\Users\\ylin\\Desktop\\ZO Spend\\"+"ZO spend.csv", index=False)

In [177]:
data.pivot_table(values="cSpend", index="Month", columns=["DSP", "Type", "Public"], 
                 aggfunc=np.sum).round().to_csv("D:\\Users\\ylin\\Desktop\\ZO Spend\\"+"ZO spend pivot.csv")

In [178]:
data.pivot_table(values="cSpend", index="Month", columns=["DSP", "Type", "Public"], aggfunc=np.sum).round()

DSP            DBM                      RocketFuel      TTD            \
Type       General             TrueView    General  General             
Public          No        Yes       N/A        Yes       No       Yes   
Month                                                                   
2016/01   462491.0  2015535.0  126240.0        NaN      NaN       0.0   
2016/02   599799.0  2321375.0  180716.0        NaN      NaN       0.0   
2016/03  1042068.0  2574432.0  279131.0        NaN      NaN       NaN   
2016/04   525837.0  2141692.0  371005.0        NaN      NaN    2615.0   
2016/05   537853.0  1718345.0  153631.0        NaN      NaN   19819.0   
2016/06   601384.0  1900652.0  286835.0        NaN      NaN   21837.0   
2016/07   470456.0  1444104.0  225570.0        NaN  19428.0   60549.0   
2016/08   685058.0  1577548.0  138528.0        NaN  29773.0  163617.0   
2016/09   843696.0  1788445.0  172688.0        NaN  37963.0   71513.0   
2016/10   784882.0  2080459.0  380387.0        NaN      3.0   45319.0   
2016/11   987119.0  1905395.0  350509.0        NaN  23429.0   98703.0   
2016/12  1018084.0  1783496.0  174375.0        NaN  91615.0   74246.0   
2017/01   442769.0  1138666.0  201930.0     5743.0  17489.0   95751.0   
2017/02   419031.0   947721.0  175352.0     6835.0  16691.0  131737.0   
2017/03   596981.0  1165207.0  139000.0     5570.0  33715.0  266813.0   
2017/04   593276.0  1188412.0  132948.0    22024.0  23500.0  234321.0   

DSP     Videology            
Type    Channel 4   General  
Public        N/A       N/A  
Month                        
2016/01       NaN       NaN  
2016/02       NaN       NaN  
2016/03       NaN       NaN  
2016/04       NaN       NaN  
2016/05  121445.0  341517.0  
2016/06  163329.0  353411.0  
2016/07  104554.0  162814.0  
2016/08  215635.0  113612.0  
2016/09  302641.0  161261.0  
2016/10  831893.0  227547.0  
2016/11  841687.0  145597.0  
2016/12  480049.0   31477.0  
2017/01  264591.0  212558.0  
2017/02  518240.0  228212.0  
2017/03  433708.0  237404.0  
2017/04  518357.0  147026.0